In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip 'drive/My Drive/Hackathon_DB (1) (1).zip' -d 'drive/My Drive'

Archive:  drive/My Drive/Hackathon_DB (1) (1).zip
replace drive/My Drive/Host.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
import os
os.chdir('drive/My Drive')

In [0]:
import pandas as pd
#alertl = pd.read_csv('drive/My Drive/AlertLevel.csv')
rvbsm = pd.read_csv('RvBsm.csv')
#summary = pd.read_csv('drive/My Drive/Summary.csv')
#eventappid = pd.read_csv('drive/My Drive/EventAppID.csv')
host = pd.read_csv('Host.csv')
#rvzone = pd.read_csv('drive/My Drive/RvZone.csv')
evtwarn = pd.read_csv('EvtWarn.csv')
#vehclass = pd.read_csv('drive/My Drive/RvBasicVehClass.csv')
#spat = pd.read_csv('Spat.csv')

In [0]:
evtwarn.head()

,Device,Trip,Time,RvDevice,NativeFlag,LocalTimeMS,PsId,RvBasicVehClass,RvRandomId,AlertLevel,...,RvLatOffset,HvLatitude,HvLongitude,HvHeading,HvElevation,HvSpeed,HvBrakeStatus,HvYawRate,HvLongitudinalAccel,HvTurnSignal
0,2004,12,48150,2494,1,80559500,1,0,1368,2,...,-4.904795,40.258686,-83.463096,110.27,295.51999,31.577778,0,0.24,-0.0836,0
1,2004,12,48250,2494,1,80560500,1,0,1368,2,...,-5.043336,40.258585,-83.462745,110.43,295.42001,31.611113,0,0.24,0.0716,0
2,2004,12,48350,2494,1,80561500,1,0,1368,2,...,-5.195783,40.258485,-83.462393,110.53,295.32001,31.622221,0,0.24,0.2074,0
3,2004,12,48450,2494,1,80562500,1,0,1368,2,...,-5.365892,40.258383,-83.462042,110.57,295.12000,31.561111,0,0.24,0.1686,0
4,2004,54,109450,22081,1,83779400,1,0,0,3,...,0.000000,40.236264,-83.366918,4.32,267.22000,6.277778,0,-0.97,0.8282,0


In [0]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [0]:
# not required for updated dataset
# evtwarn['Time']=evtwarn['Time'].str.replace(',','').astype(int)
# evtwarn['Device']=evtwarn['Device'].str.replace(',','').astype(int)
# evtwarn['Trip']=evtwarn['Trip'].str.replace(',','').astype(int)

In [0]:
evtwarn.head()

,Device,Trip,Time,RvDevice,NativeFlag,LocalTimeMS,PsId,RvBasicVehClass,RvRandomId,AlertLevel,...,RvLatOffset,HvLatitude,HvLongitude,HvHeading,HvElevation,HvSpeed,HvBrakeStatus,HvYawRate,HvLongitudinalAccel,HvTurnSignal
0,2004,12,48150,"2,494",1,"80,559,500",1,0,"1,368",2,...,-4.905,40.259,-83.463,110.27,295.52,31.578,0,0.24,-0.084,0
1,2004,12,48250,"2,494",1,"80,560,500",1,0,"1,368",2,...,-5.043,40.259,-83.463,110.43,295.42,31.611,0,0.24,0.072,0
2,2004,12,48350,"2,494",1,"80,561,500",1,0,"1,368",2,...,-5.196,40.258,-83.462,110.53,295.32,31.622,0,0.24,0.207,0
3,2004,12,48450,"2,494",1,"80,562,500",1,0,"1,368",2,...,-5.366,40.258,-83.462,110.57,295.12,31.561,0,0.24,0.169,0
4,2004,54,109450,"22,081",1,"83,779,400",1,0,0,3,...,0.000,40.236,-83.367,4.32,267.22,6.278,0,-0.97,0.828,0


In [0]:
# event data corresponding to IMA and FCW
# Since warning are continous in time, ignore warnings in a rolling period of 5s
evt_ = evtwarn[(evtwarn['EventAppId'] == 2) | (evtwarn['EventAppId']==1)]
ignore_interval = 5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
            and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1


In [0]:
# Calculate reaction times for individual warnings, max of 60s
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=60
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness.head()

,Device,Trip,ReactionTime,Time
0,2004.0,54.0,6.0,109450.0
1,2004.0,55.0,10.0,9800.0
2,2008.0,61.0,0.5,222650.0
3,2147.0,156.0,0.5,136900.0
4,2331.0,20.0,0.5,194450.0


In [0]:
# label reactiveness : less that 1sec as 1

# reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 4 if x*2>4 else x*2-1)
reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 1 if x<=0.5 else 0)
reactiveness['reactiveness']=reactiveness['reactiveness'].astype(int)

In [0]:
reactiveness['reactiveness'].value_counts()

1    100
0     31
Name: reactiveness, dtype: int64

In [0]:
# add a start time to split host
r_ = reactiveness.copy()
r_.rename(columns = {'Time':'EndTime'}, inplace = True)
r_['StartTime']= 0
for i,row in r_.iterrows():
  if (i-1 in reactiveness.index) and (reactiveness.loc[i-1]['Device']==row['Device']) and (reactiveness.loc[i-1]['Trip']==row['Trip']):
    r_['StartTime'].loc[i]=reactiveness['Time'].loc[i-1]
reactiveness = r_.copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
r_.head()

,Device,Trip,ReactionTime,EndTime,reactiveness,StartTime
0,2004.0,54.0,6.0,109450.0,0,0.0
1,2004.0,55.0,10.0,9800.0,0,0.0
2,2008.0,61.0,0.5,222650.0,1,0.0
3,2147.0,156.0,0.5,136900.0,1,0.0
4,2331.0,20.0,0.5,194450.0,1,0.0


In [0]:
# normalize dataset on the host table
host['Speed']=(host['Speed']-min(host['Speed'].values))/(max(host['Speed'].values)-min(host['Speed'].values))
host['ThrottlePosPct']=(host['ThrottlePosPct']-min(host['ThrottlePosPct'].values))/(max(host['ThrottlePosPct'].values)-min(host['ThrottlePosPct'].values))
host['SteerAngle']=(host['SteerAngle']-min(host['SteerAngle'].values))/(max(host['SteerAngle'].values)-min(host['SteerAngle'].values))
host['LongAccel']=(host['LongAccel']-min(host['LongAccel'].values))/(max(host['LongAccel'].values)-min(host['LongAccel'].values))


In [0]:
# split host
x_train = []
y_train = []
max_len = -float('inf')
features = ['BrakeStatus','Speed','ThrottlePosPct','SteerAngle','LongAccel']

for i,row in reactiveness.iterrows():
    x=(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['StartTime']) \
            & (host['Time']>row['EndTime'])][features]).values
    if len(x)>=max_len:
        max_len = len(x)
    x_train.append(x)
    y_train.append(row['reactiveness'])

In [0]:
# create same length by appending zeros
import numpy as np
x_t = []
for x in x_train:
    zeros = np.zeros((max_len-len(x),len(features)))
    x_t.append(np.append(x,zeros,axis=0))

In [0]:
# categorical classes for multiclass classification
# from keras.utils.np_utils import to_categorical   

# y_train = to_categorical(y_train, num_classes=5)

In [0]:
# convert to np arrays
x_ = np.array(x_t)
y_ = np.array(y_train)
from sklearn.utils import shuffle
x_, y_ = shuffle(x_, y_, random_state=0)

In [0]:
# def normalize(x):
#   #where x is a col
#   maxval = max(x)
#   minval = min(x)
#   x_= (x-minval)/(maxval - minval)
#   print (x_)
#   return x_

In [0]:
# x_ = np.apply_along_axis(normalize,0,x_)

In [0]:
# from scipy.stats import describe
# describe(x_)

In [0]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
from sklearn.model_selection import train_test_split

In [0]:
# binary classification lstm model
model = Sequential()
model.add(LSTM(128,input_shape=(x_.shape[1:]),activation='relu',return_sequences=True))

model.add(LSTM(128,activation='relu'))

model.add(Dense(32,activation='relu'))

# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# complie
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.2, random_state=42)

In [0]:
# Code to dublicate the 0s in the dataset. each zero is copied 3 times
x_ = []
y_ = []

for x,y in zip(x_train,y_train):
  if y==0:
    for i in range(2):
      x_.append(x)
      y_.append(y)
  x_.append(x)
  y_.append(y)
x_, y_ = shuffle(x_, y_, random_state=0)
x_train = np.array(x_)
y_train = np.array(y_)


In [0]:
# fit

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 156 samples, validate on 27 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
156/156 [==============================] - 204s 1s/sample - loss: 0.6930 - acc: 0.4744 - val_loss: 0.6943 - val_acc: 0.1852
Epoch 2/3
156/156 [==============================] - 195s 1s/sample - loss: 0.6923 - acc: 0.5000 - val_loss: 0.6956 - val_acc: 0.1852
Epoch 3/3
156/156 [==============================] - 196s 1s/sample - loss: 0.6917 - acc: 0.5000 - val_loss: 0.6952 - val_acc: 0.1852


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.2, random_state=42)
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Train on 104 samples, validate on 27 samples
Epoch 1/5
104/104 [==============================] - 154s 1s/sample - loss: 0.6928 - acc: 0.6154 - val_loss: 0.6865 - val_acc: 0.8148
Epoch 2/5
104/104 [==============================] - 154s 1s/sample - loss: 0.6868 - acc: 0.7500 - val_loss: 0.6776 - val_acc: 0.8148
Epoch 3/5
104/104 [==============================] - 154s 1s/sample - loss: 0.6774 - acc: 0.7500 - val_loss: 0.6652 - val_acc: 0.8148
Epoch 4/5
104/104 [==============================] - 154s 1s/sample - loss: 0.6683 - acc: 0.7500 - val_loss: 0.6447 - val_acc: 0.8148
Epoch 5/5
104/104 [==============================] - 154s 1s/sample - loss: 0.6487 - acc: 0.7500 - val_loss: 0.5986 - val_acc: 0.8148


Multiple Event App Ids

In [0]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 2) | (evtwarn['EventAppId']==1)]
ignore_interval = 5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
            and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1


In [0]:
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=60
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness.head()

,Device,Trip,ReactionTime,Time
0,2004.0,54.0,6.0,109450.0
1,2004.0,55.0,10.0,9800.0
2,2008.0,26.0,60.0,92500.0
3,2008.0,61.0,0.5,222650.0
4,2017.0,301.0,60.0,59350.0


In [0]:
# label reactiveness : less that 1sec as 1

# reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 4 if x*2>4 else x*2-1)
reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 1 if x<=0.5 else 0)
reactiveness['reactiveness']=reactiveness['reactiveness'].astype(int)

In [0]:
reactiveness['reactiveness'].value_counts()

1    102
0     46
Name: reactiveness, dtype: int64

In [0]:
# split host
r_ = reactiveness.copy()
r_.rename(columns = {'Time':'EndTime'}, inplace = True)
r_['StartTime']= 0
for i,row in r_.iterrows():
  if (i-1 in reactiveness.index) and (reactiveness.loc[i-1]['Device']==row['Device']) and (reactiveness.loc[i-1]['Trip']==row['Trip']):
    r_['StartTime'].loc[i]=reactiveness['Time'].loc[i-1]
reactiveness = r_.copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
r_.head()

,Device,Trip,ReactionTime,EndTime,reactiveness,StartTime
0,2004.0,54.0,6.0,109450.0,0,0.0
1,2004.0,55.0,10.0,9800.0,0,0.0
2,2008.0,61.0,0.5,222650.0,1,0.0
3,2147.0,156.0,0.5,136900.0,1,0.0
4,2331.0,20.0,0.5,194450.0,1,0.0


In [0]:
# split host
x_train = []
y_train = []
max_len = -float('inf')
features = ['BrakeStatus','Speed','ThrottlePosPct','SteerAngle','LongAccel']

for i,row in reactiveness.iterrows():
    x=(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['StartTime']) \
            & (host['Time']>row['EndTime'])][features]).values
    if len(x)>=max_len:
        max_len = len(x)
    x_train.append(x)
    y_train.append(row['reactiveness'])

In [0]:
# create same length by appending zeros
import numpy as np
x_t = []
for x in x_train:
    zeros = np.zeros((max_len-len(x),len(features)))
    x_t.append(np.append(x,zeros,axis=0))

In [0]:
# convert to np arrays
x_ = np.array(x_t)
y_ = np.array(y_train)
from sklearn.utils import shuffle
x_, y_ = shuffle(x_, y_, random_state=0)

In [0]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
from sklearn.model_selection import train_test_split

In [0]:
# binary classification lstm model
model = Sequential()
model.add(LSTM(128,input_shape=(x_.shape[1:]),activation='relu',return_sequences=True))

model.add(LSTM(128,activation='relu'))

model.add(Dense(32,activation='relu'))

# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# complie
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.2, random_state=42)

In [0]:
# Code to dublicate the 0s in the dataset. each zero is copied 3 times
x_ = []
y_ = []

for x,y in zip(x_train,y_train):
  if y==0:
    for i in range(2):
      x_.append(x)
      y_.append(y)
  x_.append(x)
  y_.append(y)
x_, y_ = shuffle(x_, y_, random_state=0)
x_train = np.array(x_)
y_train = np.array(y_)


In [0]:
# fit

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 190 samples, validate on 30 samples
Epoch 1/3
190/190 [==============================] - 236s 1s/sample - loss: 0.6927 - acc: 0.5684 - val_loss: 0.6968 - val_acc: 0.3333
Epoch 2/3
190/190 [==============================] - 234s 1s/sample - loss: 0.6895 - acc: 0.5684 - val_loss: 0.7036 - val_acc: 0.3333
Epoch 3/3
190/190 [==============================] - 234s 1s/sample - loss: 0.6838 - acc: 0.5684 - val_loss: 0.7304 - val_acc: 0.3333


In [0]:
p0 = (len(y_test)-y_test.sum())/len(y_test)
print ('% of 0s:',p0)

% of 0s: 0.3333333333333333


In [0]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 2) | (evtwarn['EventAppId']==1)]
ignore_interval = 5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
            and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1


In [0]:
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=60
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness.head()

,Device,Trip,ReactionTime,Time
0,2004.0,54.0,6.0,109450.0
1,2004.0,55.0,10.0,9800.0
2,2008.0,26.0,60.0,92500.0
3,2008.0,61.0,0.5,222650.0
4,2017.0,301.0,60.0,59350.0


In [0]:
# label reactiveness : less that 1sec as 1

# reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 4 if x*2>4 else x*2-1)
reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 1 if x<=0.5 else 0)
reactiveness['reactiveness']=reactiveness['reactiveness'].astype(int)

In [0]:
reactiveness['reactiveness'].value_counts()

1    102
0     46
Name: reactiveness, dtype: int64

In [0]:
# split host
r_ = reactiveness.copy()
r_.rename(columns = {'Time':'EndTime'}, inplace = True)
r_['StartTime']= 0
for i,row in r_.iterrows():
  if (i-1 in reactiveness.index) and (reactiveness.loc[i-1]['Device']==row['Device']) and (reactiveness.loc[i-1]['Trip']==row['Trip']):
    r_['StartTime'].loc[i]=reactiveness['Time'].loc[i-1]
reactiveness = r_.copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
r_.head()

,Device,Trip,ReactionTime,EndTime,reactiveness,StartTime
0,2004.0,54.0,6.0,109450.0,0,0.0
1,2004.0,55.0,10.0,9800.0,0,0.0
2,2008.0,26.0,60.0,92500.0,0,0.0
3,2008.0,61.0,0.5,222650.0,1,0.0
4,2017.0,301.0,60.0,59350.0,0,0.0


In [0]:
# split host
x_train = []
y_train = []
max_len = -float('inf')
features = ['BrakeStatus','Speed','ThrottlePosPct','SteerAngle','LongAccel']

for i,row in reactiveness.iterrows():
    x=(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['StartTime']) \
            & (host['Time']>row['EndTime'])][features]).values
    if len(x)>=max_len:
        max_len = len(x)
    x_train.append(x)
    y_train.append(row['reactiveness'])

In [0]:
# create same length by appending zeros
import numpy as np
x_t = []
for x in x_train:
    zeros = np.zeros((max_len-len(x),len(features)))
    x_t.append(np.append(x,zeros,axis=0))

In [0]:
# convert to np arrays
x_ = np.array(x_t)
y_ = np.array(y_train)
from sklearn.utils import shuffle
x_, y_ = shuffle(x_, y_, random_state=0)

In [0]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
from sklearn.model_selection import train_test_split

In [0]:
# binary classification lstm model
model = Sequential()
model.add(LSTM(128,input_shape=(x_.shape[1:]),activation='relu',return_sequences=True))

model.add(LSTM(128,activation='relu'))

model.add(Dense(128,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(32,activation='relu'))

# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# complie
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.2, random_state=42)

In [0]:
# Code to dublicate the 0s in the dataset. each zero is copied 3 times
x_ = []
y_ = []

for x,y in zip(x_train,y_train):
  if y==0:
    for i in range(1):
      x_.append(x)
      y_.append(y)
  x_.append(x)
  y_.append(y)
x_, y_ = shuffle(x_, y_, random_state=0)
x_train = np.array(x_)
y_train = np.array(y_)


In [0]:
# fit

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 154 samples, validate on 30 samples
Epoch 1/3
154/154 [==============================] - 196s 1s/sample - loss: 0.6934 - acc: 0.4156 - val_loss: 0.6929 - val_acc: 0.6667
Epoch 2/3
154/154 [==============================] - 194s 1s/sample - loss: 0.6932 - acc: 0.4416 - val_loss: 0.6922 - val_acc: 0.6667
Epoch 3/3
154/154 [==============================] - 196s 1s/sample - loss: 0.6928 - acc: 0.5455 - val_loss: 0.6914 - val_acc: 0.6667


In [0]:
p0 = (len(y_test)-y_test.sum())/len(y_test)
print ('% of 0s:',p0)

% of 0s: 0.3333333333333333


In [0]:
evt_ = evtwarn
ignore_interval = 5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
            and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1


In [0]:
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=60
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness.head()

,Device,Trip,ReactionTime,Time
0,2004.0,12.0,60.0,48150.0
1,2004.0,54.0,6.0,109450.0
2,2004.0,55.0,10.0,9800.0
3,2004.0,55.0,1.5,10650.0
4,2004.0,119.0,2.0,129650.0


In [0]:
# label reactiveness : less that 1sec as 1

# reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 4 if x*2>4 else x*2-1)
reactiveness['reactiveness'] = reactiveness['ReactionTime'].map(lambda x: 1 if x<=0.5 else 0)
reactiveness['reactiveness']=reactiveness['reactiveness'].astype(int)

In [0]:
reactiveness['reactiveness'].value_counts()

0    172
1    127
Name: reactiveness, dtype: int64

In [0]:
# split host
r_ = reactiveness.copy()
r_.rename(columns = {'Time':'EndTime'}, inplace = True)
r_['StartTime']= 0
for i,row in r_.iterrows():
  if (i-1 in reactiveness.index) and (reactiveness.loc[i-1]['Device']==row['Device']) and (reactiveness.loc[i-1]['Trip']==row['Trip']):
    r_['StartTime'].loc[i]=reactiveness['Time'].loc[i-1]
reactiveness = r_.copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
r_.head()

,Device,Trip,ReactionTime,EndTime,reactiveness,StartTime
0,2004.0,12.0,60.0,48150.0,0,0.0
1,2004.0,54.0,6.0,109450.0,0,0.0
2,2004.0,55.0,10.0,9800.0,0,0.0
3,2004.0,55.0,1.5,10650.0,0,9800.0
4,2004.0,119.0,2.0,129650.0,0,0.0


In [0]:
# split host
x_train = []
y_train = []
max_len = -float('inf')
features = ['BrakeStatus','Speed','ThrottlePosPct','SteerAngle','LongAccel']

for i,row in reactiveness.iterrows():
    x=(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['StartTime']) \
            & (host['Time']>row['EndTime'])][features]).values
    if len(x)>=max_len:
        max_len = len(x)
    x_train.append(x)
    y_train.append(row['reactiveness'])

In [0]:
# create same length by appending zeros
import numpy as np
x_t = []
for x in x_train:
    zeros = np.zeros((max_len-len(x),len(features)))
    x_t.append(np.append(x,zeros,axis=0))

In [0]:
# convert to np arrays
x_ = np.array(x_t)
y_ = np.array(y_train)
from sklearn.utils import shuffle
x_, y_ = shuffle(x_, y_, random_state=0)

In [0]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
from sklearn.model_selection import train_test_split

In [0]:
# binary classification lstm model
model = Sequential()
model.add(LSTM(128,input_shape=(x_.shape[1:]),activation='relu',return_sequences=True))

model.add(LSTM(128,activation='relu'))

model.add(Dense(128,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(32,activation='relu'))

# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# complie
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.2, random_state=42)

In [0]:
# Code to dublicate the 0s in the dataset. each zero is copied 3 times
x_ = []
y_ = []

for x,y in zip(x_train,y_train):
  if y==0:
    for i in range(1):
      x_.append(x)
      y_.append(y)
  x_.append(x)
  y_.append(y)
x_, y_ = shuffle(x_, y_, random_state=0)
x_train = np.array(x_)
y_train = np.array(y_)


In [0]:
# fit

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 239 samples, validate on 60 samples
Epoch 1/3
239/239 [==============================] - 341s 1s/sample - loss: 0.6930 - acc: 0.5188 - val_loss: 0.6923 - val_acc: 0.5667
Epoch 2/3
239/239 [==============================] - 336s 1s/sample - loss: 0.6913 - acc: 0.5816 - val_loss: 0.6903 - val_acc: 0.5667
Epoch 3/3
239/239 [==============================] - 337s 1s/sample - loss: 0.6880 - acc: 0.5816 - val_loss: 0.6872 - val_acc: 0.5667


In [0]:
p0 = (len(y_test)-y_test.sum())/len(y_test)
print ('% of 0s:',p0)

% of 0s: 0.5666666666666667


In [0]:
# imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
from sklearn.model_selection import train_test_split

In [0]:
# binary classification lstm model
model = Sequential()
model.add(LSTM(128,input_shape=(x_.shape[1:]),activation='relu',return_sequences=True))

model.add(LSTM(128,activation='relu'))

# model.add(Dense(128,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(32,activation='relu'))

# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# complie
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# fit

model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Train on 239 samples, validate on 60 samples
Epoch 1/3
239/239 [==============================] - 340s 1s/sample - loss: 0.6931 - acc: 0.5021 - val_loss: 0.6929 - val_acc: 0.5667
Epoch 2/3
239/239 [==============================] - 336s 1s/sample - loss: 0.6923 - acc: 0.5816 - val_loss: 0.6914 - val_acc: 0.5667
Epoch 3/3
239/239 [==============================] - 335s 1s/sample - loss: 0.6904 - acc: 0.5816 - val_loss: 0.6898 - val_acc: 0.5667


In [0]:
p0 = (len(y_test)-y_test.sum())/len(y_test)
print ('% of 0s:',p0)